# Testing Instruction Fine-Tuned LLMS
This notebook is made to offer means for testing the hand-crafted prompting method on a selection of Instruction Fine-Tuned LLMs.

## Disclaimer
This notebook is made to recreate Plaza-Del-Arco, F., Nozza, D., & Hovy, D. (2023). Respectful or Toxic? Using Zero-Shot Learning with Language Models to Detect Hate Speech. Retrieved from https://aclanthology.org/2023.woah-1.6.pdf

## Instruction Fine-Tuned Model
In this section, we try the instruction fine-tuned models with their prompt template.

In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification
import pandas as pd
from tqdm.notebook import tqdm

### Load Model and Tokenizer
You cann load either Flan-T5 or mT-0. Run the cell of the desired model. By default if you run the notebook, Flan-T5 is loaded.

In [136]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xl")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl")

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [3]:
if not model:
    model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/mt0-base")
    tokenizer = AutoTokenizer.from_pretrained("bigscience/mt0-base")

### Setting the Prompt Template

In [4]:
prompt_template = "Classify this text as hate or non-hate. Text:"
output_indicator = "Answer:"

### Loading Test Data

In [5]:
df = pd.read_csv('./data/preprocessed_data.csv')
df.describe()

,HS
count,145556.000000
mean,0.365894
std,0.481681
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [6]:
labels = {
    "0": "Non-Hate",
    "1": "Hate Speech",
}

### Concatenate Prompt Template to Input Samples

In [7]:
def concat_prompt_template(df_column):
    return df_column.apply(lambda x: f"{prompt_template} {x}. {output_indicator}")

In [8]:
df['input'] = concat_prompt_template(df['text'])
text_data = df['input'].astype("str").tolist()

### Tokenize and Predict

In [9]:
inputs = tokenizer(text_data, return_tensors="pt", padding=True, truncation=True)

In [10]:
model.to('cuda')

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [11]:
inputs.to('cuda')

{'input_ids': tensor([[4501, 4921,   48,  ...,    0,    0,    0],
        [4501, 4921,   48,  ...,    0,    0,    0],
        [4501, 4921,   48,  ...,    0,    0,    0],
        ...,
        [4501, 4921,   48,  ...,    0,    0,    0],
        [4501, 4921,   48,  ...,    0,    0,    0],
        [4501, 4921,   48,  ...,    0,    0,    0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')}

In [12]:
batch_size = 16
def batch_inference(data_list):
    decoded = list()
    for i in tqdm(range(0, len(data_list)-batch_size, batch_size)):
        inputs = tokenizer(data_list[i: i+batch_size], return_tensors="pt", padding=True, truncation=True)
        inputs = {key: value.to('cuda:0') for key, value in inputs.items()}
        sequences = model.generate(**inputs, do_sample=True, min_length=0, max_length=10, temperature=0.001)
        decoded.extend(tokenizer.batch_decode(sequences, skip_special_tokens=True))
    return decoded
decoded = batch_inference(text_data)

  0%|          | 0/9097 [00:00<?, ?it/s]

In [22]:
labels

{'0': 'Non-Hate', '1': 'Hate Speech'}

In [23]:
df

,text,HS,input
0,yes indeed she sort of reminds me of the elder...,0,Classify this text as hate or non-hate. Text: ...
1,the trans women reading this tweet right now i...,0,Classify this text as hate or non-hate. Text: ...
2,question these broads who criticize america wh...,0,Classify this text as hate or non-hate. Text: ...
3,it is about time for all illegals to go back t...,0,Classify this text as hate or non-hate. Text: ...
4,for starters bend over the one in pink and kic...,1,Classify this text as hate or non-hate. Text: ...
...,...,...,...
145551,you unfollowed me fuck you pussy,0,Classify this text as hate or non-hate. Text: ...
145552,stfu bitch and you go make some satanic music ...,1,Classify this text as hate or non-hate. Text: ...
145553,honey as a fellow white chick let me tell you ...,0,Classify this text as hate or non-hate. Text: ...
145554,i hate bitches who talk about niggaz with kids...,1,Classify this text as hate or non-hate. Text: ...


In [26]:
filename = 'flan_t5_results.csv'
file_object = open(filename, 'w')
# print('omar')
for i, out in enumerate(decoded):
    file_object.write(out)
    file_object.write(',')
    file_object.write(labels[str(df["HS"][i])])
    file_object.write('\n')
file_object.close()

In [96]:
results = pd.read_csv(filename)
results.columns = [ "truth","output"]
results.head()

,truth,output
0,Non-hate,Non-Hate
1,Non-hate,Non-Hate
2,Non-hate,Non-Hate
3,Hate,Hate Speech
4,Non-hate,Non-Hate


In [97]:
results

,truth,output
0,Non-hate,Non-Hate
1,Non-hate,Non-Hate
2,Non-hate,Non-Hate
3,Hate,Hate Speech
4,Non-hate,Non-Hate
...,...,...
145546,Non-hate,Non-Hate
145547,Non-hate,Non-Hate
145548,Non-hate,Hate Speech
145549,Hate,Hate Speech


### Answer Mapping

In [98]:
filename = 'flan_t5_results.csv'
results = pd.read_csv(filename)
results.columns = ["truth","output"]
results.head()

,truth,output
0,Non-hate,Non-Hate
1,Non-hate,Non-Hate
2,Non-hate,Non-Hate
3,Hate,Hate Speech
4,Non-hate,Non-Hate


### Qualitative Analysis
First of all, we want to know how many answers were not mapped to either "Hate" or "Non-Hate"

In [99]:
def count_non_correct_outputs(series):
    return series.apply()

In [100]:
results['isCorrect'] = results['output'].apply(lambda x: x.lower() == 'hate' or x.lower() == 'non-hate')

In [102]:
results.head()

,truth,output,isCorrect
0,Non-hate,Non-Hate,True
1,Non-hate,Non-Hate,True
2,Non-hate,Non-Hate,True
3,Hate,Hate Speech,False
4,Non-hate,Non-Hate,True


In [103]:
correctCount = results[results.isCorrect == True]
len(correctCount)

92296

We got almost half of the samples with correct output, so we begin and map those.

In [104]:
correctCount.head()

,truth,output,isCorrect
0,Non-hate,Non-Hate,True
1,Non-hate,Non-Hate,True
2,Non-hate,Non-Hate,True
4,Non-hate,Non-Hate,True
5,hate,Non-Hate,True


Convert all offensive labels to Non-hate

In [105]:
# correctCount['truthModified'] = correctCount['truth'].apply(lambda x: 'Non-hate' if x.lower() == 'offensive' else x)

In [106]:
correctCount.head()

,truth,output,isCorrect
0,Non-hate,Non-Hate,True
1,Non-hate,Non-Hate,True
2,Non-hate,Non-Hate,True
4,Non-hate,Non-Hate,True
5,hate,Non-Hate,True


Now we count the correct answers

In [108]:
correctCount['isCorrectAns'] = correctCount.apply(lambda x: x.output.lower() == x.truth.lower(), axis=1)
correctAnsCount = len(correctCount[correctCount.isCorrectAns == True])
correctAnsCount

/tmp/ipykernel_13815/2007881201.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  correctCount['isCorrectAns'] = correctCount.apply(lambda x: x.output.lower() == x.truth.lower(), axis=1)


57055

### Mapping the answer classes to either 1 or 0

In [126]:
def mapAnswers(answer):
    if answer.lower() == 'non-hate':
        return 0
    elif answer.lower() == 'hate' or answer.lower() == 'hate speech':
        return 1
    else: return None

results['outputLabel'] = results['output'].apply(mapAnswers)
results['truthLabel'] = results['truth'].apply(mapAnswers)

In [127]:
results['truthLabel']

0         0
1         0
2         0
3         1
4         0
         ..
145546    0
145547    0
145548    0
145549    1
145550    1
Name: truthLabel, Length: 145551, dtype: int64

In [128]:
results['outputLabel']

0         0
1         0
2         0
3         1
4         0
         ..
145546    0
145547    0
145548    1
145549    1
145550    0
Name: outputLabel, Length: 145551, dtype: int64

In [129]:
results = results.dropna()

In [130]:
len(results)

145551

In [82]:
!pip install scikit-learn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [131]:
from sklearn.metrics import f1_score

In [134]:
score = f1_score(correctCount['truthLabel'], correctCount['outputLabel'], average = 'macro')

In [135]:
score

0.38201953786717197

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
import torch

print(torch.version.cuda)
torch.cuda.memory_allocated()


In [ ]:
import sys

sys.executable

In [ ]:
!pip install --upgrade jupyter ipython


In [138]:
!git status

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   instruction_fine_tuned_lms.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	flan_t5_results.csv
	../testing-playground/

no changes added to commit (use "git add" and/or "git commit -a")


In [139]:
! git add instruction_fine_tuned_lms.ipynb flan_t5_results.csv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [140]:
!git commit -m"added flant5 results and notebook uodates"

[main d29f209] added flant5 results and notebook uodates
 2 files changed, 146714 insertions(+), 237 deletions(-)
 create mode 100644 recreating-benchmark/flan_t5_results.csv


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [141]:
! git push

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Username for 'https://github.com': ^C
